In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [ ]:
file_path = "data/hand_landmarks_data.csv"  
data = pd.read_csv(file_path)

In [23]:
data.columns

Index(['x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'x5', 'y5', 'x6', 'y6', 'x7', 'y7',
       'x8', 'y8', 'x9', 'y9', 'x10', 'y10', 'x11', 'y11', 'x12', 'y12', 'x13',
       'y13', 'x14', 'y14', 'x15', 'y15', 'x16', 'y16', 'x17', 'y17', 'x18',
       'y18', 'x19', 'y19', 'x20', 'y20', 'x21', 'y21', 'label'],
      dtype='object')

In [24]:
X = data.drop(columns=[col for col in data.columns if col.startswith("label")])  
y = data[[col for col in data.columns if col.startswith("label")]]  

In [25]:
y.columns

Index(['label'], dtype='object')

In [26]:
X.columns

Index(['x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'x5', 'y5', 'x6', 'y6', 'x7', 'y7',
       'x8', 'y8', 'x9', 'y9', 'x10', 'y10', 'x11', 'y11', 'x12', 'y12', 'x13',
       'y13', 'x14', 'y14', 'x15', 'y15', 'x16', 'y16', 'x17', 'y17', 'x18',
       'y18', 'x19', 'y19', 'x20', 'y20', 'x21', 'y21'],
      dtype='object')

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200, 300], 
    'max_depth': [10, 20, 30, None],  
    'min_samples_split': [2, 5, 10],   
    'min_samples_leaf': [1, 2, 4]      
}
grid_search_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)
best_rf = grid_search_rf.best_estimator_
print(f"✅ the best Random Forest: {grid_search_rf.best_params_}")

c:\Users\YOUSSEF\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


✅ the best Random Forest: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 30, 50, 100], 
    'gamma': [0.001, 0.01, 0.045, 0.1, 0.5], 
    'kernel': ['rbf'] 
}
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)


c:\Users\YOUSSEF\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


✅ the best Random Forest: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [49]:
best_svm = grid_search.best_estimator_
print(f"✅ The best SVM parameters: {grid_search.best_params_}")


✅ The best SVM parameters: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}


In [53]:
model = SVC(kernel="rbf",C=100, gamma=0.001,random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("acc = ",acc)

c:\Users\YOUSSEF\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


acc =  0.9593345656192237


In [58]:
param_grid = {
    'n_estimators': [50, 100, 200, 300], 
    'max_depth': [10, 20, 30, None],     
    'min_samples_split': [2, 5, 10],      
    'min_samples_leaf': [1, 2, 4]         
}
grid_search_etc = GridSearchCV(ExtraTreesClassifier(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_etc.fit(y_test, y_pred)

print(f"✅ The best parameters: {grid_search_etc.best_params_}")
print(f"📈 The best accuracy: {grid_search_etc.best_score_:.4f}")


✅ The best parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
📈 The best accuracy: 0.9593


In [ ]:


param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],  
    'weights': ['uniform', 'distance'], 
    'metric': ['euclidean', 'manhattan', 'minkowski']  
}
grid_search_knn = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_knn.fit(y_test, y_pred)
print(f"✅ Best parameters: {grid_search_knn.best_params_}")
print(f"📈 Best accuracy: {grid_search_knn.best_score_:.4f}")

✅ Best parameters: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}
📈 Best accuracy: 0.9593
